### Persiapan Data

In [4]:
# EDA
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Text Cleaning
import neattext.functions as nfx

# Machine Learning
# Estimators
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# Transformers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

### Data Cleaning

### Machine Learning

In [5]:
# Membaca File
data_emotion = pd.read_csv("Dataset Babak Penyisihan.csv")
data_emotion

,label,tweet
0,fear,Lebih menyeramkan kalo punya grup WA keluarga ...
1,joy,"Hepibesdey canteeeekk [USERNAME] panjang umur,..."
2,joy,Jumat lalu nonton inih! Selain tertarik settin...
3,joy,Nyante dulu.. Sruput kopi biar tambah bijak...
4,love,Happy annyversarry yg pertama kalinya .. Trsra...
...,...,...
5148,NaN,Aku termasuk orang yang tidak habis pikir deng...
5149,NaN,"Melawan sibuk Selasamu, jangan patah semangat ..."
5150,NaN,Momen di mana kamu merasa begitu mendalam terh...
5151,NaN,Tapi tentunya secara ini Bekasi ada hal-hal yg...


In [15]:
# Memisahkan Data Mentah dan Predict
# data_emotion.dropna(subset = "label")
for i in data_emotion.index :
    if (data_emotion.loc[i,"label"] is None) :
        data_emotion.drop(i, index = True)
        print("oke")
cek_notKosong = data_emotion.isnull().sum()
cek_notKosong

label    1000
tweet       2
dtype: int64